#Kernel Python 3.12

## Librerias a usar

In [295]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors



## Conexion y carga de datos

In [296]:
#Conexion para pruebas rápidas
ratings_df = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
peliculas_df = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

In [297]:
nombrePeliculas = dict(zip(peliculas_df['movieId'], peliculas_df['title']))

movie_ratings = ratings_df.merge(peliculas_df, on='movieId')
movie_ratings['title'].value_counts()[0:-100]

title
Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
                                   ... 
Tyrannosaur (2011)                    1
This Must Be the Place (2011)         1
Miss Representation (2011)            1
Starsuckers (2009)                    1
Girl Walks Into a Bar (2011)          1
Name: count, Length: 9619, dtype: int64

## Matriz dispersa

In [298]:
def MatrizDispersa(df):
    usuarios = df['userId'].nunique()
    peliculas = df['movieId'].nunique()

    mapUsuario = dict(zip(np.unique(df["userId"]), list(range(usuarios))))
    mapPelicula = dict(zip(np.unique(df["movieId"]), list(range(peliculas))))
    
    mapUsuarioTranspuesta = dict(zip(list(range(usuarios)), np.unique(df["userId"])))
    mapPeliculaTranspuesta = dict(zip(list(range(peliculas)), np.unique(df["movieId"])))
    
    indiceUsuario = [mapUsuario[i] for i in df['userId']]
    indicePelicula = [mapPelicula[i] for i in df['movieId']]

    Matriz = csr_matrix((df["rating"], (indiceUsuario,indicePelicula)), shape=(usuarios,peliculas))
    
    return Matriz, mapUsuario, mapPelicula, mapUsuarioTranspuesta, mapPeliculaTranspuesta

Matriz, mapUsuario, mapPelicula, mapUsuarioTranspuesta, mapPeliculaTranspuesta = MatrizDispersa(ratings)

In [299]:
def KNNPelicula(indicePelicula, Matriz, mapPelicula, mapPeliculaTranspuesta, k, metric='cosine'):
    Matriz = Matriz.T
    indiceVecinos = []
    
    indPelicula = mapPelicula[indicePelicula]
    listaPelicula = Matriz[indPelicula]
    if isinstance(listaPelicula, (np.ndarray)):
        listaPelicula = listaPelicula.reshape(1,-1)
    # use k+1 since kNN output includes the movieId of interest
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric = metric)
    kNN.fit(Matriz)
    vecino = kNN.kneighbors(listaPelicula, return_distance=False) #False
    for i in range(0,k):
        n = vecino.item(i)
        indiceVecinos.append(mapPeliculaTranspuesta[n])
    indiceVecinos.pop(0)
    return indiceVecinos

In [300]:
pelicula = int(input("Ingrese numero de usuario"))
peliculasSugeridas = KNNPelicula(pelicula, Matriz, mapPelicula, mapPeliculaTranspuesta, k=10)
peliculasSugeridas

[3114, 480, 780, 260, 356, 364, 1210, 648, 1265]

## Mostrar recomendaciones

In [301]:
print(f"Segun la pelicula que te gustó {nombrePeliculas[pelicula]}: \n te sugerimos las siguientes")
for i in peliculasSugeridas:
    print(nombrePeliculas[i])

Segun la pelicula que te gustó Toy Story (1995): 
 te sugerimos las siguientes
Toy Story 2 (1999)
Jurassic Park (1993)
Independence Day (a.k.a. ID4) (1996)
Star Wars: Episode IV - A New Hope (1977)
Forrest Gump (1994)
Lion King, The (1994)
Star Wars: Episode VI - Return of the Jedi (1983)
Mission: Impossible (1996)
Groundhog Day (1993)
